## Tratar Feiras Livres

In [4]:
import geopandas as gpd
import os

# Caminho do arquivo .gpkg
input_gpkg = "SIRGAS_GPKG_TEMA_ABASTECIMENTO_FEIRAS_LIVRES.gpkg"

# Ler o arquivo .gpkg com GeoPandas
gdf = gpd.read_file(input_gpkg)

# Exportar para GeoJSON
output_geojson = "FEIRAS_LIVRES.geojson"
gdf.to_file(output_geojson, driver="GeoJSON")

print(f"Arquivo GeoJSON salvo em: {output_geojson}")


Arquivo GeoJSON salvo em: FEIRAS_LIVRES.geojson


## Converter crs do arquivo geojson

In [5]:
# Função para processar GeoJSONs e salvá-los no formato desejado
def processar_geojsons(arquivos_geojson, pasta_saida='dados_processados', crs_destino=4326, formato='GeoJSON'):
    # Verificar se a pasta de saída existe, senão, criar
    if not os.path.exists(pasta_saida):
        os.makedirs(pasta_saida)
    
    geojson_list = []
    
    for arquivo in arquivos_geojson:
        # Carregar o GeoJSON
        gdf = gpd.read_file(arquivo)
        
        # Verificar se o CRS já está no formato de destino
        if gdf.crs.to_epsg() != crs_destino:
            # Transformar para coordenadas globais (EPSG:4326) somente se necessário
            gdf = gdf.to_crs(epsg=crs_destino)
        
        # Verificar e limpar geometrias inválidas ou nulas
        gdf = gdf[gdf.is_valid].dropna(subset=['geometry'])
        
        # Nome do arquivo de saída
        nome_saida = os.path.join(pasta_saida, os.path.basename(arquivo).replace('.geojson', f'_processado.{formato.lower()}'))
        
        # Salvar o arquivo no formato desejado
        if formato.lower() == 'geojson':
            gdf.to_file(nome_saida, driver='GeoJSON')
        elif formato.lower() == 'shapefile':
            gdf.to_file(nome_saida, driver='ESRI Shapefile')
        
        # Adicionar ao geojson_list
        geojson_list.append((arquivo, gdf))
    
    return geojson_list

In [6]:
# Lista de arquivos GeoJSON
arquivos_geojson = [
    'FEIRAS_LIVRES.geojson',
]

# Processar os GeoJSONs e salvá-los
geojson_list = processar_geojsons(arquivos_geojson, formato='GeoJSON')

## Converter para arquivo .xlsx

In [7]:
import pandas as pd

# Caminho do arquivo GeoJSON
geojson_file = "FEIRAS_LIVRES.geojson"

# Ler o arquivo GeoJSON com GeoPandas
gdf = gpd.read_file(geojson_file)

# Extrair coordenadas de latitude e longitude
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

# Remover a coluna geometry (opcional)
gdf = gdf.drop(columns='geometry')

# Converter o GeoDataFrame para um DataFrame do Pandas
df = pd.DataFrame(gdf)

# Exportar para um arquivo Excel (.xlsx)
output_excel = "Feiras Livres.xlsx"
df.to_excel(output_excel, index=False)

## Analisar o arquivo .xlsx

In [9]:

# Caminho para o arquivo .xlsx
excel_file = "Feiras Livres.xlsx"

# Ler o arquivo Excel
df = pd.read_excel(excel_file)

# Exibir as primeiras linhas do arquivo (para ter uma visão geral)
print(df.head())

# Exibir o nome das colunas
print("Colunas:", df.columns)

# Exibir o resumo estatístico das colunas numéricas
print("\nResumo estatístico das colunas numéricas:")
print(df.describe())

# Exibir informações gerais sobre o DataFrame
print("\nInformações gerais:")
print(df.info())


         eq_nome                            eq_enderec  eq_bairro  eq_cep  \
0       MANDAQUI  RUA GENERAL JOSE ALMEIDA BOTELHO S/N        NaN     NaN   
1     SANTA INES                 RUA ANA DE BARROS S/N        NaN     NaN   
2  SANTO ANTONIO          PRACA  SALVADOR TOLEZANO S/N        NaN     NaN   
3    JOAO PESSOA             RUA RIBEIRAO DO SALTO S/N        NaN     NaN   
4        HERISON                       RUA HERISON S/N        NaN     NaN   

   eq_horario  eq_telefon  eq_regiao  eq_distrit  eq_subpref   eq_esfera  \
0         NaN         NaN        NaN         NaN         NaN  SEM ESFERA   
1         NaN         NaN        NaN         NaN         NaN  SEM ESFERA   
2         NaN         NaN        NaN         NaN         NaN  SEM ESFERA   
3         NaN         NaN        NaN         NaN         NaN  SEM ESFERA   
4         NaN         NaN        NaN         NaN         NaN  SEM ESFERA   

       eq_classe   eq_rede   eq_tipo  longitude   latitude  
0  FEIRAS_LIVRES  S

In [10]:
# Exibir a contagem de valores únicos por coluna
print("\nContagem de valores únicos por coluna:")
print(df.nunique())


Contagem de valores únicos por coluna:
eq_nome       844
eq_enderec    926
eq_bairro       0
eq_cep          0
eq_horario      0
eq_telefon      0
eq_regiao       0
eq_distrit      0
eq_subpref      0
eq_esfera       1
eq_classe       1
eq_rede         1
eq_tipo         1
longitude     936
latitude      936
dtype: int64


In [11]:
# Remover colunas que estão completamente vazias
df = df.dropna(axis=1, how='all')

# Exibir colunas após a remoção
print("Colunas após remover vazias:", df.columns)

# Se quiser salvar o DataFrame modificado em um novo arquivo Excel
df.to_excel("Feiras Livres.xlsx", index=False)

Colunas após remover vazias: Index(['eq_nome', 'eq_enderec', 'eq_esfera', 'eq_classe', 'eq_rede', 'eq_tipo',
       'longitude', 'latitude'],
      dtype='object')
